# EuroSAT Data Analysis

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from pathlib import Path
from collections import Counter

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

%matplotlib inline

## Load dataset

In [ ]:
def load_eurosat_dataset(data_dir="data"):
    data_path = Path(data_dir)

    # Get all class directories
    class_dirs = [d for d in data_path.iterdir() if d.is_dir()]
    class_names = sorted([d.name for d in class_dirs])

    print(f"Found {len(class_names)} classes: {class_names}")

    images = []
    labels = []

    # Load images from each class
    for class_idx, class_name in enumerate(class_names):
        class_path = data_path / class_name
        image_files = list(class_path.glob("*.jpg")) + list(class_path.glob("*.png"))

        print(f"Loading {len(image_files)} images from {class_name}...")

        for img_path in image_files:
            try:
                # Load image
                img = Image.open(img_path)
                img_array = np.array(img)

                # Store image and label
                images.append(img_array)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")

    # Convert to numpy arrays
    data = np.array(images)
    labels = np.array(labels)

    print(f"\nDataset loaded successfully!")
    print(f"Total images           : {len(data)}")
    print(f"Data shape             : {data.shape}")
    print(f"Labels shape           : {labels.shape}")

    return data, labels, class_names


# Load the dataset
data, labels, class_names = load_eurosat_dataset("data")

## Preview dataset

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(10, 10))

random_indices = np.random.choice(len(data), size=25, replace=False)

for i, ax in enumerate(axes.flat):
    idx = random_indices[i]
    img = data[idx]
    label = labels[idx]
    class_name = class_names[label]

    # Display image
    ax.imshow(img)
    ax.set_title(class_name, fontsize=16, pad=3)
    ax.axis("off")

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.suptitle(
    "Random Sample: 25 Images",
    fontsize=24,
    fontweight="bold",
)
plt.show()

In [ ]:
fig, axes = plt.subplots(10, 5, figsize=(10, 25))

sample_count_per_class = 5
selected_indices = []

for class_idx in range(len(class_names)):
    class_mask = labels == class_idx
    class_indices = np.where(class_mask)[0]
    selected = np.random.choice(
        class_indices, size=sample_count_per_class, replace=False
    )
    selected_indices.extend(selected)

# Plot images
for i, ax in enumerate(axes.flat):
    idx = selected_indices[i]
    img = data[idx]
    label = labels[idx]
    class_name = class_names[label]

    # Display image
    ax.imshow(img)
    ax.set_title(class_name, fontsize=16, pad=3)
    ax.axis("off")

plt.tight_layout(rect=[0, 0, 1, 0.975])
plt.suptitle("Balanced Sample: 5 Images per Class", fontsize=24, fontweight="bold")
plt.show()

## Dataset statistics

In [ ]:
# Number of samples per class
class_counts = Counter(labels)

for idx, class_name in enumerate(class_names):
    count = class_counts[idx]
    percentage = (count / len(labels)) * 100
    print(f"{class_name:25s}: {count:4d} images ({percentage:4.2f}%)")

In [ ]:
# Images statistics
print(f"Total number of images      : {len(data)}")
print(f"Number of classes           : {len(class_names)}")
print(f"Image shape                 : {data[0].shape}")
print(f"Data array shape            : {data.shape}")
print(f"Data array dtype            : {data.dtype}")
print(f"Labels array shape          : {labels.shape}")
print(f"Labels array dtype          : {labels.dtype}")

In [ ]:
# Class distribution
fig, ax = plt.subplots(figsize=(12, 6))
counts = [class_counts[i] for i in range(len(class_names))]
bars = ax.bar(class_names, counts, color=sns.color_palette("husl", len(class_names)))
ax.set_xlabel("Class", fontsize=12, fontweight="bold")
ax.set_ylabel("Number of Images", fontsize=12, fontweight="bold")
ax.set_title("Class Distribution in EuroSAT Dataset", fontsize=14, fontweight="bold")
plt.xticks(rotation=45, ha="right")
ax.grid(axis="y", alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{int(height)}",
        ha="center",
        va="bottom",
        fontsize=10,
    )

plt.tight_layout()
plt.show()

## Image analysis

In [ ]:
# Image statistics
print(f"Minimum pixel value       : {data.min()}")
print(f"Maximum pixel value       : {data.max()}")
print(f"Mean pixel value          : {data.mean():.2f}")
print(f"Std pixel value           : {data.std():.2f}")
print(f"Median pixel value        : {np.median(data):.2f}")

In [ ]:
# Per-channel statistics (assuming RGB)
if data.shape[-1] == 3:
    channels = ["Red", "Green", "Blue"]
    for i, channel in enumerate(channels):
        print(f"\n{channel} Channel:")
        print(f"  Min       : {data[..., i].min()}")
        print(f"  Max       : {data[..., i].max()}")
        print(f"  Mean      : {data[..., i].mean():.2f}")
        print(f"  Std       : {data[..., i].std():.2f}")

In [ ]:
# Brightness
# https://stackoverflow.com/questions/596216/formula-to-determine-perceived-brightness-of-rgb-color
brightness = 0.299 * data[..., 0] + 0.587 * data[..., 1] + 0.114 * data[..., 2]
print(f"Mean brightness           : {brightness.mean():.2f}")
print(f"Std brightness            : {brightness.std():.2f}")
print(f"Min brightness            : {brightness.min():.2f}")
print(f"Max brightness            : {brightness.max():.2f}")

In [ ]:
# Contrast (std of pixel values per image)
# https://stackoverflow.com/questions/52094592/calculate-contrast-of-a-color-imagergb
contrast = data.std(axis=(1, 2, 3))
print(f"Mean contrast             : {contrast.mean():.2f}")
print(f"Std contrast              : {contrast.std():.2f}")
print(f"Min contrast              : {contrast.min():.2f}")
print(f"Max contrast              : {contrast.max():.2f}")

In [ ]:
# Statistics per class
stats_data = []
for idx, class_name in enumerate(class_names):
    class_mask = labels == idx
    class_images = data[class_mask]

    # Calculate brightness for this class
    class_brightness = (
        0.299 * class_images[..., 0]
        + 0.587 * class_images[..., 1]
        + 0.114 * class_images[..., 2]
    )

    # Calculate contrast for this class
    class_contrast = class_images.std(axis=(1, 2, 3))

    stats_data.append(
        {
            "Class": class_name,
            "Count": class_mask.sum(),
            "Mean Pixel": class_images.mean(),
            "Std Pixel": class_images.std(),
            "Mean Brightness": class_brightness.mean(),
            "Std Brightness": class_brightness.std(),
            "Mean Contrast": class_contrast.mean(),
            "Std Contrast": class_contrast.std(),
            "Min Pixel": class_images.min(),
            "Max Pixel": class_images.max(),
        }
    )

stats_df = pd.DataFrame(stats_data)
print(stats_df.round(2).to_string(index=False))

## Color analysis

In [ ]:
# Color distribution histogram
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
colors = ["red", "green", "blue"]
channels = ["Red", "Green", "Blue"]

for i, (ax, color, channel) in enumerate(zip(axes, colors, channels)):
    ax.hist(data[..., i].flatten(), bins=50, color=color, alpha=0.7, edgecolor="black")
    ax.set_xlabel("Pixel Value", fontsize=12, fontweight="bold")
    ax.set_ylabel("Frequency", fontsize=12, fontweight="bold")
    ax.set_title(f"{channel} Channel Distribution", fontsize=13, fontweight="bold")
    ax.grid(axis="y", alpha=0.3)
    ax.set_xlim(0, 255)

plt.tight_layout()
plt.show()

In [ ]:
# Brightness and Contrast distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Brightness
axes[0].hist(brightness.flatten(), bins=50, color="gray", alpha=0.7, edgecolor="black")
axes[0].set_xlabel("Brightness Value", fontsize=12, fontweight="bold")
axes[0].set_ylabel("Frequency", fontsize=12, fontweight="bold")
axes[0].set_title("Brightness Distribution (Overall)", fontsize=14, fontweight="bold")
axes[0].grid(axis="y", alpha=0.3)

# Contrast
axes[1].hist(contrast, bins=50, color="purple", alpha=0.7, edgecolor="black")
axes[1].set_xlabel("Contrast Value", fontsize=12, fontweight="bold")
axes[1].set_ylabel("Frequency", fontsize=12, fontweight="bold")
axes[1].set_title("Contrast Distribution (Overall)", fontsize=14, fontweight="bold")
axes[1].grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Per-class color distribution
n_classes = len(class_names)
n_cols = 3
n_rows = (n_classes + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, class_name in enumerate(class_names):
    ax = axes[idx]
    class_mask = labels == idx
    class_images = data[class_mask]

    # Plot RGB histograms for this class
    for i, (color, channel) in enumerate(
        zip(["red", "green", "blue"], ["R", "G", "B"])
    ):
        ax.hist(
            class_images[..., i].flatten(),
            bins=50,
            color=color,
            alpha=0.5,
            label=channel,
            density=True,
        )

    ax.set_xlabel("Pixel Value", fontsize=10)
    ax.set_ylabel("Density", fontsize=10)
    ax.set_title(f"{class_name}", fontsize=11, fontweight="bold")
    ax.legend()
    ax.grid(axis="y", alpha=0.3)
    ax.set_xlim(0, 255)

# Hide unused subplots
for idx in range(n_classes, len(axes)):
    axes[idx].axis("off")

plt.tight_layout()
plt.suptitle("Color Distribution per Class", fontsize=16, fontweight="bold", y=1.002)
plt.show()

In [ ]:
# Brightness distribution per class
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, class_name in enumerate(class_names):
    ax = axes[idx]
    class_mask = labels == idx
    class_images = data[class_mask]

    # Calculate brightness for this class
    class_brightness = (
        0.299 * class_images[..., 0]
        + 0.587 * class_images[..., 1]
        + 0.114 * class_images[..., 2]
    )

    ax.hist(
        class_brightness.flatten(), bins=50, color="gray", alpha=0.7, edgecolor="black"
    )
    ax.set_xlabel("Brightness", fontsize=10)
    ax.set_ylabel("Frequency", fontsize=10)
    ax.set_title(f"{class_name}", fontsize=11, fontweight="bold")
    ax.grid(axis="y", alpha=0.3)

# Hide unused subplots
for idx in range(n_classes, len(axes)):
    axes[idx].axis("off")

plt.tight_layout()
plt.suptitle(
    "Brightness Distribution per Class", fontsize=16, fontweight="bold", y=1.002
)
plt.show()

In [ ]:
# Contrast distribution per class
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, class_name in enumerate(class_names):
    ax = axes[idx]
    class_mask = labels == idx
    class_images = data[class_mask]

    # Calculate contrast for this class
    class_contrast = class_images.std(axis=(1, 2, 3))

    ax.hist(
        class_contrast.flatten(), bins=50, color="purple", alpha=0.7, edgecolor="black"
    )
    ax.set_xlabel("Contrast", fontsize=10)
    ax.set_ylabel("Frequency", fontsize=10)
    ax.set_title(f"{class_name}", fontsize=11, fontweight="bold")
    ax.grid(axis="y", alpha=0.3)

# Hide unused subplots
for idx in range(n_classes, len(axes)):
    axes[idx].axis("off")

plt.tight_layout()
plt.suptitle("Contrast Distribution per Class", fontsize=16, fontweight="bold", y=1.002)
plt.show()